In [ ]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

if os.getcwd().endswith("notebooks"):
    os.chdir("../..")

from ptseries.algorithms.binary_solvers import BinaryBosonicSolver
from tutorial_notebooks.utils.max_cut_utils import separate_in_two_subsets

# QUBO and The Max-Cut problem


In this notebook, we will cover the following:
- Introduction to *QUBO* tasks
- Introduction to the *max cut problem*
- Description of how to pose the *max cut problem* as a QUBO task
- Setup of the BBS algorithm to solve the *max cut problem* as a *QUBO* task

We will assume basic knowledge of how the setup of the BBS algorithm works. If you are still unsure of this, see the `optimisation_intro.ipynb` notebook first.

## An Introduction to QUBO Problems

A common formulation of binary optimization problems is as a quadratic unconstrained binary optimization (QUBO) problem. This formulation involves minimizing the quantity $x^{\top}Qx$ where $Q$ is a real symmetric matrix which completely defines the problem and $x\in \{ 0, 1\}^M$ a binary string. We want to find the binary string which achieves the minimal value for this expression.

To build intuition, lets consider a simple QUBO problem of size 2. We will give labels to the entries of $Q$:
$$Q = \left(\begin{matrix}
q_1 & q_2\\
q_2 & q_3
\end{matrix}\right)$$
Note that the matrix must be symmetric, and so we have the top right and bottom left entries equal. Now, lets consider an arbitrary length 2 binary string as a vector $x = (x_0 ,x_1)$. We will now expand out the value we want to minimise:
$$x^{\top}Qx = q_1 x_0^2 + 2q_2 x_0 x_1 + q_3 x_1^2 = q_1 x_0 + 2q_2 x_0 x_1 + q_3 x_1$$
We get rid of the squared terms as the variables are equal to 0 or 1 so $x_i^2 = x_i$. The reason for the name *QUBO* should be more clear now. We are minimising a linear sum of products of binary variables, with the products being at most *quadratic*. The problem is *unconstrained* since we are considering all possible bit strings of the appropriate length. When using the BBS algorithm, we divide into QUBO and non-QUBO problems since the inner workings of the algorithm is more efficient for problems posed as QUBO tasks.

 As a concrete example of a QUBO problem we consider the Max-Cut problem. Given a graph, the objective of the Max-Cut problem is to separate all nodes into two sets such that the number of edges between these sets is maximized. Here, we will select a simple graph with 6 nodes on which to demonstrate how to solve a max cut problem using the BBS algorithm.


## The Max Cut Problem
The max cut problem is a graph theory optimisation task. We have a set of nodes connected by edges. We want to divide the nodes into two sets, such that the number of edges we need to remove (cut) to ensure both sets are completely seperated is maximised. In other words, we want to find a  division of the nodes into two sets such that the number of edges between the two sets of nodes is maximised.

Let's consider a simple size 6 graph and visualise the problem.

In [ ]:
# Defining the structure of the graph we consider
G = nx.Graph()
edge_list = [(0, 1), (0, 2), (0, 5), (1, 3), (1, 4), (2, 4), (2, 5), (3, 4), (3, 5)]
G.add_edges_from(edge_list)
pos = nx.spring_layout(G, seed=7)  # positions for all nodes - seed for reproducibility

# Plotting a diagram of the graph
fig, ax = plt.subplots(figsize=(8, 6))
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="blue", ax=ax)
nx.draw_networkx_edges(G, pos, edgelist=edge_list, width=5, ax=ax)
nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif", ax=ax)

ax.set_axis_off()
ax.margins(0.08)
plt.tight_layout()
plt.show()

For this graph, with its small size, we can find a solution by inspection. In this case, taking, say, nodes 1,2,5 to be one set and 0,3,4 to be the other gives us a division with number of cutting edges equal to 7. We plot this division, with the dotted lines representing cut edges, and the two node colours representing the two sets.


In [ ]:
separate_in_two_subsets(G, [1, 2, 5], [0, 3, 4])

In fact, this solution isn't unique for this particular graph. If we took one set consisting of nodes 1,2,3 and the other of nodes 0,4,5 we would also get a cut consisting of 7 edges.

## Max-Cut formulation as a QUBO problem
We want to formulate this task as a QUBO problem. In other words, we need a way to encode a choice of subsets as binary strings, and then a real symmetric matrix $Q$ such that $x^{T}Qx$ is equal to the number of cut edges between the subset choice corresponding to $x$. First, we define how we correspond subset choices to binary strings. For a size $m$ graph, we will consider length $m$ binary strings $x_0 x_1...x_{m-1}$ with the correspondence:

$$
x_i = \left\{
    \begin{array}{ll}
        0 \quad \text{if node } i \text{ is in subset A} \\
        1 \quad \text{if node } i \text{ is in subset B}
    \end{array}
\right.
$$
As such, any binary string will correspond to a valid choice of subset division. Now, we want to have a quadratic *indicator function* that is 1 if nodes $i$ and $j$ are in different subsets, and the edge $(i,j)$ is in the graph. This corresponds to this edge having to be cut. We can achieve this with the following expression:

$$
x_i + x_j -2x_i x_j = \left\{
    \begin{array}{ll}
        0 \quad \text{if} \quad(x_i, x_j) \quad \text{are in the same subset} \\
        1 \quad \text{if} \quad(x_i, x_j) \quad \text{are in different subset}
    \end{array}
\right.
$$
We now need to define our real symmetric matrix $Q$ to be such that these expressions are added together for each edge $(i,j)$ present in the graph. But this would give us our QUBO cost function being equal to the cut associated with a choice of subsets. As QUBO is a minimsation task, we need to take the inverse of this. In other words, we need:

$$
x^{\top}Qx = -\sum_{(x_i,x_j) \in E} x_i + x_j -2x_i x_j
$$
Expanding out the expression for the QUBO cost value, we can equate this with our target cost function above to see which values we need to take for $Q$:
$$x^{T}Qx = \sum_{i=1}^m\sum_{j=1}^m Q_{ij}x_i x_j$$
With a bit of algebra and comparing terms, we find that we need:
$$Q_{ii} = (-1) \times  \text{no. edges that node }i\text{ has} \\ Q_{ij} =     \begin{cases}
      1 & \text{if edge} (i,j) \text{ is in the graph}\\
      0 & \text{otherwise}
    \end{cases} $$


## Solving Max-Cut with the BBS Algorithm

With the background maths done, we have posed the max-cut problem in QUBO formulation. Now, all that remains to do is to define the appropriate matrix $Q$ and feed it into the BBS algorithm object.

In [ ]:
# Define the matrix Q used for QUBO
Q = np.zeros((6, 6))
for i, j in G.edges:
    Q[i, i] += -1
    Q[j, j] += -1
    Q[i, j] += 1
    Q[j, i] += 1

print(Q)

Note that, unlike for when we consider non-QUBO optimisation tasks, we don't need to feed in a cost function into the BBS algorithm. All we need to do is give the BBS algorithm the numpy array Q. With this being passed in place of a cost function, the algorithm will automatically consider the corresponding QUBO problem, and the back end will act appropriately. We set up  and train the BBS algorithm below.

In [ ]:
bbs = BinaryBosonicSolver(pb_dim=6, objective=Q)
bbs.train(learning_rate=1e-1, updates=80, print_frequency=20)

We look at the solution that the BBS algorithm has come to. Since there are several solutions that are just as good as each other, the solution found here may not be the same as the solution shown above. As long as it has a cut size of 7, it is an optimal solution.

In [ ]:
# pick one of the optimal solutions encountered by the algorithm
opt_solution = bbs.config_min_encountered

# Displays the optimal solution
nodes_in_set1 = [idx for idx in range(len(opt_solution)) if opt_solution[idx] == 0]
nodes_in_set2 = [idx for idx in range(len(opt_solution)) if opt_solution[idx] == 1]

separate_in_two_subsets(G, nodes_in_set1, nodes_in_set2)

The BBS algorithm has found an optimal solution with cut size 7! We have introduced QUBO formulation, shown how certian problems can be posed as QUBO tasks and set up the BBS algorithm to solve such a task.